### Course Announcements

**Due Sunday (11:59 PM)**
- D6
- Q7 
- Checkpoint #1 (discuss Data section)
- Weekly Project Survey (optional)

# Text Analysis

- **Sentiment Analysis**
    - tokenization
    - stop words
    - stemming
- **TF-IDF**
    - Bag of Words
    - term frequency
    - inverse document frequency
- Tools: `nltk`

In [1]:
# pandas and matplotlib setup
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (17, 7)
plt.rcParams.update({'font.size': 14})
import seaborn as sns

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'

import warnings
warnings.filterwarnings('ignore')

#import natural language toolkit
import nltk

# download stopwords & punkt
nltk.download('stopwords')
nltk.download('punkt')

#### Reminder: **Natural Language Processing** is a whole field of study.

Like most topics in this course, there are many courses solely focused on the appropriate analysis of text. We'll cover the general concepts in this course, but know you're missing lots of important details.

## Natural Language Toolkit (`nltk`)

For more details on using the functionality within this package, check out the [NLTK Book](http://www.nltk.org/book/).

0. Preface
1. Language Processing and Python
2. Accessing Text Corpora and Lexical Resources
3. Processing Raw Text
4. Writing Structured Programs
5. Categorizing and Tagging Words 
6. Learning to Classify Text
7. Extracting Information from Text
8. Analyzing Sentence Structure
9. Building Feature Based Grammars
10. Analyzing the Meaning of Sentences 
11. Managing Linguistic Data
12. Afterword: Facing the Language Challenge

[VADER](https://github.com/cjhutto/vaderSentiment) is a particularly helpful tool/lexicon when working with sentiments expressed in social media (tweets, online reviews, etc.)

Its functionality is available through `nltk`, so we'll download the vader lexicon for use later in this notebook.

In [ ]:
# get lexicon we'll be working with today
nltk.download('vader_lexicon') 

## The Data

In [2]:
quarters = ['Wi24', 'Wi21', 'Fa20', 'Wi20', 'Sp20', 'Sp19']

In [3]:
def read_data(quarter):
    '''read data in from specified quarter, extract columns of interest 
    and add a column indicating quarter from which data originated'''
    
    df = pd.read_csv('https://raw.githubusercontent.com/shanellis/datasets/master/COGS108_feedback_' + quarter + '.csv')
    df = df[['enjoyed_most', 'enjoyed_least']]
    df['quarter'] = quarter
    
    return df

In [4]:
# read in data for all quarters
df = pd.DataFrame()

for quarter in quarters:
    qtr = read_data(quarter)
    df = pd.concat([df, qtr], ignore_index=True)
    
df

,enjoyed_most,enjoyed_least,quarter
0,"The pace is moderate, easy to follow but not b...",We have to work in a group. personally speakin...,Wi24
1,I enjoy exploring data science more using Python.,"I don't really like when labs, lecture, and as...",Wi24
2,"guest lectures, being able to hear real life e...",weekly lecture quizzes,Wi24
3,"Assignment, there is a step by step guide how ...","Assignment, because its also time consuming.",Wi24
4,Coding,Group members not responding,Wi24
...,...,...,...
2290,the fact that the professor codes in class liv...,the fact that the group partners were assigned...,Sp19
2291,No exams,Team project,Sp19
2292,The availability of office hours and style of ...,I don't know much about Python and it is reall...,Sp19
2293,The availability of office hours and style of ...,I don't know much about Python and it is reall...,Sp19


## Describe & Explore

We'll quickly describe and explore the data to see what information we have before moving on to Text Analysis.

### Data Considerations

- duplicate responses?
- PIDs for individuals in the class (typos?)
- missingness?
- reflect reality?

In [ ]:
# how many from each quarter?
df.value_counts('quarter')

Note: Response Rates
- Spring 2019: 384/826 (46%)
- Winter 2020: 295/444 (66%)
- Spring 2020: 397/475 (84%)
- Fall 2020: 321/447 (72%)
- Winter 2021: 314/438 (72%)
- **Winter 2024: 584/701 (83%)**

### Missingness

In [ ]:
# how many nonresponses
df.isnull().sum()

We see that there are more nonresponses in the `enjoyed_least` category than the `enjoyed_most` category. So, more people left what they enjoyed least blank than they did what they enjoyed most.

In [ ]:
# how does that look by quarter?
null_most = df.groupby('quarter')['enjoyed_most'].apply(lambda x: x.isna().sum())
null_least = df.groupby('quarter')['enjoyed_least'].apply(lambda x: x.isna().sum())

print(null_most, null_least)

We also see a decrease in Wi21 on. This is when I started requiring these questions (b/c I shortened the survey overall). 

#### Previous Quarters

Typically, there are a few people who have what they enjoy least but don't have an enjoy most....but often these students' feedback is of particular interest to me.

In [ ]:
# overall
check_least = df[df['enjoyed_most'].isnull() & df['enjoyed_least'].notnull()]
list(check_least['enjoyed_least'])

Missing data causes a problem in `nltk`, so we either get rid of individuals who didn't respond to both, or we can replace their missing data with 'No response', knowing that this text will be included in the analysis now.

In [ ]:
def fill_no_response(df):
    '''replace missing data in enjoyed_most/least series with string No response'''
    
    df['enjoyed_most'] = df['enjoyed_most'].fillna('No response')
    df['enjoyed_least'] = df['enjoyed_least'].fillna('No response')

In [ ]:
# fill NAs with string 'No response'
fill_no_response(df)

## Quick checks: Words of interest



In [ ]:
def check_word_freq(df, word):
    """checks for frequenccy of word specified in most and least enjoyed responses"""
    
    # calculate proportion within quarter
    word_most = df[df['enjoyed_most'].str.contains(word, case=False, na=False)]
    proportion_most = word_most.groupby('quarter').size() / df.groupby('quarter').size()
    
    word_least = df[df['enjoyed_least'].str.contains(word, case=False, na=False)]
    proportion_least = word_least.groupby('quarter').size() / df.groupby('quarter').size()
    
    out = combined_df = pd.concat([proportion_most, proportion_least], keys=['most', 'least'], axis=1)

    return out

#### Assignment

In [ ]:
## check for assignment
check_word_freq(df, 'assignment')

#### Project

In [ ]:
## check for project in free text
check_word_freq(df, 'project')

In [ ]:
## check for group in free text
check_word_freq(df, 'group')

#### Quizzes

In [ ]:
check_word_freq(df, 'quiz')

#### Labs

In [ ]:
check_word_freq(df, 'lab')

## Sentiment Analysis

We get a quick snapshot of what's going on in COGS 108, but we really want to understand the details. To do this, analyzing the sentiment of the text is a good next step.

#### Step 1: Tokenization

Tokenization is the first step in analyzing text. 

1. Aquire text of interest
2. Break text down (tokenize) into smaller chunks (i.e. words, bigrams, sentences, etc.)

A **token** is a single entity - think of it as a building block of language.

### Tokenization Example

Here we demonstrate what a tokenized single response looks like.

In [ ]:
# import regex word tokenizer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
tokenized_word = tokenizer.tokenize(df.loc[0,'enjoyed_most'])
print(tokenized_word)

#### Tokenize COGS108 data

Using that concept we'll tokenize the words in the enjoyed_most and `enjoyed_least` columns for the data in our COGS108 data.

In [ ]:
# tokenize most and least responses
df['most_token'] = df['enjoyed_most'].apply(tokenizer.tokenize) 
df['least_token'] = df['enjoyed_least'].apply(tokenizer.tokenize) 
df.head()

#### Step 2: Stop Words

**Stop words** are words that are of less interest to your analysis. 

For example, you wouldn't expect the following words to be important: is, am, are, this, a, an, the, etc.

By removing stopwords, you can lower the computational burden, focusing on only the words of interest.

To do so in `nltk`, you need to create a list of stopwords and filter them from your tokens.


In [ ]:
# import stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# look at stop words
print(stop_words)

### Stop Words Example

Here we compare a sentence after tokenization to one that has been tokenized _and had stop words removed_.

In [ ]:
# example of removing stop words
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
print("Tokenized Sentence:", tokenized_word)
print("Filtered Sentence:", filtered_sent)

#### Remove Stop Words: COGS108 data

Using that idea, we can go ahead and remove stop words from our tokenized most and least liked tokenized data.

In [ ]:
# remove stop words
df['most_stop'] = df['most_token'].apply(lambda x: [item for item in x if item not in stop_words])
df['least_stop'] = df['least_token'].apply(lambda x: [item for item in x if item not in stop_words])
df.head()

#### Step 3: Lexicon Normalization (**Stemming**)

In language, many different words come from the same root word. 

For example, "intersection", "intersecting", "intersects", and "intersected" are all related to the common root word - "intersect".

**Stemming** is how linguistic normalization occurs - it reduces words to their root words (and chops off additional things like 'ing') - all of the above words would be reduced to their common stem "intersect."






### Stemming Example

After tokenization and removing stop words, we can get the stem for all tokens (words) in our dataset.

In [ ]:
# Stemming
from nltk.stem import PorterStemmer

ps = PorterStemmer()

stemmed_words=[]
for w in filtered_sent:
    stemmed_words.append(ps.stem(w))

print("Filtered Sentence:", filtered_sent)
print("Stemmed Sentence:", stemmed_words)

#### Stemming: COGS108 data

Here, we obtain the stem (root word) for all tokens in our dataset.

In [ ]:
df['most_stem'] = df['most_stop'].apply(lambda x: [ps.stem(y) for y in x])
df['least_stem'] = df['least_stop'].apply(lambda x: [ps.stem(y) for y in x])
df.head()

#### Step 4: Frequency Distribution

It can be helpful to get a sense of which words are most frequent in our dataset.

In [ ]:
# get series of all most and least liked words after stemming
# note that "No Response" is still being included in the analysis
most = df['most_stem'].apply(pd.Series).stack()
least = df['least_stem'].apply(pd.Series).stack()

`FreqDist` calculates the frequency of each word in the text and we can plot the most frequent words.

In [ ]:
from nltk.probability import FreqDist
import string

# calculation word frequency
fdist_most = FreqDist(most)
fdist_least = FreqDist(least)

# remove punctuation counts
for punc in string.punctuation:
    del fdist_most[punc]
    del fdist_least[punc]

In [ ]:
# Frequency Distribution Plot - top 20
# for words in what students like most
fdist_least.plot(20, cumulative=False);

#### Step 5: Sentiment Analysis!

**Sentiment Analysis** quantifies the content, idea, beliefs and opinions conveyed in text. 

Two general approaches:

1. **Lexicon-based** - count number of words in a text belonging to each sentiment (positive, negative, happy, angry, etc.)
2. **Machine learning-based** - develop a classification model on pre-labeled data



### Sentiment Example

To get a measure of overall sentiment in our text, we'll compare our text to the VADER lexicon.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
analyser = SentimentIntensityAnalyzer()

VADER handles:

- capitalization (great vs GREAT) & punctuation (exclamation makes more positive!)
- emojis and emoticons
- degree modifiers (extremely good vs. marginally good)
- contractions and conjunctions (but signals shift)

`pos` + `neg` + `neu` = 1

**`compound`** score - metric that calculates sum of all the lexicon ratings and normalizes between -1 (most extreme negative) and +1 (most extreme positive)
- positive: `compound` >= 0.05 
- neutral: -0.05 < `compound` < 0.05
- negative : `compound` <= -0.05

In [ ]:
analyser.polarity_scores("The class is super cool.")

In [ ]:
analyser.polarity_scores("The class is not super cool.")

In [ ]:
analyser.polarity_scores("The class is NOT super cool!")

#### Sentiment Analysis: COGS108 data

Here, we will calculate the sentiment of each most liked and least liked student response from the survey.

In [ ]:
# get list of the 'sentences' (responses) from each individual
most_list = list(df[df['quarter'] == 'Wi24']['enjoyed_most'].values)
least_list = list(df[df['quarter'] == 'Wi24']['enjoyed_least'].values)

In [ ]:
# create function that will output dataframe 
# that stores sentiment information
def get_sentiments(input_list):
    
    output = pd.DataFrame(columns=['neg', 'neu', 'pos', 'compound', 'sentence'])

    for sentence in input_list:
        ss = analyser.polarity_scores(sentence)
        ss['sentence'] = sentence
        output = output.append(ss, ignore_index=True)

    return output

In [ ]:
# get sentiment measures
least_sentiments = get_sentiments(least_list)
most_sentiments = get_sentiments(most_list)

#### Sentiment Analysis: COGS108 data output

After calculating the sentiment of each response, we can look at the output of each.

In [ ]:
# let's get rid of those no response values here
most_sentiments = most_sentiments[most_sentiments['sentence'] != 'No response']
least_sentiments = least_sentiments[least_sentiments['sentence'] != 'No response']

In [ ]:
# take a look at the output
least_sentiments.sort_values(by='neg', ascending=False).head(10)

In [ ]:
# take a look at the output
most_sentiments.sort_values(by='pos', ascending=False).head(10)

#### Sentiment Analysis: COGS108 data - `describe`

To get an overall sense of the values stored in each of these dataframes, we can use `describe`.

In [ ]:
most_sentiments.describe()

In [ ]:
least_sentiments.describe()

#### Sentiment Analysis: COGS108 data - plotting

We can compare the distribution of the `compound` metric between the two analyses.

In [ ]:
most_sentiments['compound'].plot.density(label='most')
least_sentiments['compound'].plot.density(label='least')
plt.legend()
plt.xlabel('Compound Sentiment Scores')
plt.xlim(-1,1);

In [ ]:
# include label for boxplot
most_sentiments['which'] = 'most'
least_sentiments['which'] = 'least'
# concatenate data frames together
compound_out = pd.concat([most_sentiments, least_sentiments])
compound_out.head()

In [ ]:
# plot compound by resonse type
sns.boxplot(data=compound_out, x='which', y='compound')
plt.xlabel('response');

Probably unsurprisingly, the overall sentiment of what students like tends to be more positive than what students like less. 

Probably not surprising given the data and question on the survey. But, let's dig deeper into these data moving beyond sentiment analysis...

## TF-IDF

Term Frequency - Inverse Document Frequency (**TF-IDF**) sets out to identify the tokens most unique to your document of interest (relative to all documents in your corpus). 

**Term Frequency (TF)** - counts the number of words (tokens) occurring in each document.

**Inverse Document Frequency (IDF)** - weights the word by their relative frequency across documents. 

$$IDF_{word} = log(\frac{\# documents}{\# \ documents\_containing\_word})$$

$$TF-IDF = TF \times IDF$$

words with a high TF-IDF are those with high frequency in one document & relatively low frequency in other documents

For our purposes, our **corpus** will be students' responses to what they like most and least about COGS108.

We'll treat this as **two separate documents**:
1. What students like most
2. What students like least

### Bag of Words (BoW) approach

Converts the text into a co-occurrence matrix across documents within the corpus.

To do this, let's get our text ready.

We're going to make sure all our words are lower case, remove punctuation from each, and then provide the text (`corpus`) to `TfidfVectorizer`.

In [ ]:
import string 

# lowercase text
least = list(map(str.lower, least_list))
most = list(map(str.lower, most_list))

# remove punctuation
for c in string.punctuation:
    least = str(least).replace(c, "")
    most = str(most).replace(c, "")

# get list of two documents together
corpus = [str(least), str(most)]

### Calculate TF-IDF

With our text ready for analysis, it's time to calculate TF-IDF

To start our TF-IDF analysis, we'll first **create a `TfidfVectorizer` object to transform our text data into vectors.**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# create vectorizer
tfidf = TfidfVectorizer(sublinear_tf=True,
                        analyzer='word',
                        max_features=2000,
                        tokenizer=word_tokenize,
                        stop_words=stop_words)

#### TF-IDF: COGS108 data - calculation

Here, we use our vectorizer to calculate TF-IDF across the words in our word matrix.

In [ ]:
# calculate TF-IDF
cogs_tfidf = pd.DataFrame(tfidf.fit_transform(corpus).toarray())
cogs_tfidf.columns = tfidf.get_feature_names()
cogs_tfidf = cogs_tfidf.rename(index={0:'least', 1:'most'})

#### TF-IDF: COGS108 data - output

If we just want to look at the word most uniuqe in each document...

In [ ]:
most_unique = cogs_tfidf.idxmax(axis=1) 
most_unique

Alternatively, we can sort by the set or words most unique to each document:

In [ ]:
cogs_tfidf.sort_values(by='most', axis=1, ascending=False)

In [ ]:
cogs_tfidf.sort_values(by='least', axis=1, ascending=False)

**Sentiment Analysis** and **TF-IDF** are really helpful when analyzing documents and corpuses of text.

But, what if, from the text itself we wanted to predict whether or not the text was likely a 'most' liked or a 'least' liked comment? We'll discuss how to do this in the coming **machine learning** lectures!